In [6]:
pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=7780ffc028a74a3594a51003a5eed66257313e8bf202b69bc1a3304dd5dac1fd
  Stored in directory: c:\users\dudco\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.1
    Uninstalling numpy-1.20.1:
      Successfully uninstalled numpy-1.20.1
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0



  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached protobuf-3.17.3-cp38-cp38-win_amd64.whl (909 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached h5py-3.1.0-cp38-cp38-win_amd64.whl (2.7 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached grpcio-1.34.1-cp38-cp38-win_amd64.whl (2.9 MB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached google_auth-1.34.0-py2.py3-none-any.whl (152 kB)
  Using cached

In [13]:
#감성분석

from konlpy.tag import Okt  #형태소분석기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences   #패딩 설정 
from tensorflow.keras.models import load_model  #모델 불러오기
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

okt = Okt()    #형태소 분석
tokenizer = Tokenizer(19417, oov_token = 'OOV')  #oov_token 특수한 값(oov) 처리 - 사전에 토큰화되지 않은 단어가 생략되는 것을 ovv로 처리
with open('wordIndex.json') as json_file:
  word_index = json.load(json_file)
  tokenizer.word_index = word_index  #단어와 숫자의 key-value를 포함하는 딕셔너리 반환


loaded_model = load_model('best_model.h5')   #감성분석 모델 불러오기
def sentiment_predict(new_sentence):
    print(new_sentence)
    max_len = 30
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    #stopwords: 불용어 리스트 -> 배제
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩  maxlen: 시퀀스(많이 ) 최대길이 제한
    score = float(loaded_model.predict(pad_new)) # 예측
    return score  #긍정/부정에 대한 점수
    if score >= 0.5:
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score*100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1-score)*100))


In [14]:
#네이버 영화 리뷰_크롤링

from bs4 import BeautifulSoup
import urllib.request as req

page_num = 1
while True:
    code = req.urlopen("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=189069&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}".format(page_num))
    soup = BeautifulSoup(code, "html.parser")
    comment = soup.select("li > div.score_reple > p > span")
    if len(comment) == 0:
        break
    for i in comment:
        i = i.text.strip()
        if i == "관람객":
            continue

        #print(i)
        result = sentiment_predict(i)
        print(result)
        print("-------------------------------------")
    page_num += 1
    # 양이 많으니 페이지 10개만 
    if page_num == 10:
        break

박정민이 왜 포스터, 예고에 없는 지 알았음. 어쩜 그리 연기를 잘 하는지
0.7835091352462769
-------------------------------------
박정민은진짜다박정민은진짜다인정하자
0.6522318720817566
-------------------------------------
목숨과바꾼 가족사랑  그리고 액션 특히 박정민 배우님에 충격적인 변신..좋았어요
0.9959299564361572
-------------------------------------
역시 박정민!!!!조연이아니라 나에겐주연  강철비2보려다 조연으로박정민이나온다해서바로선택한영화인데완전나의신의하수였다?
0.9554305076599121
-------------------------------------
박정민이 가장 강렬했음
0.9572926163673401
-------------------------------------
와 이거 감독판으로도 나왔으면 좋겠다
0.945243775844574
-------------------------------------
역시 믿고 보는 배우 황정민. 이정재.박정민 이영화는 꼭봐야한다!!!올해 최고의 작품 강추
0.9988682270050049
-------------------------------------
황정민이랑 이정재 같이 나오는 영화는 다 재밌어
0.954969048500061
-------------------------------------
몰입감 긴장감으로 시간 순삭ㅋㅋㅋ 존잼
0.9751993417739868
-------------------------------------
스토리가 없어요 정말이에요
0.600499153137207
-------------------------------------
영화 세줄요약 1. 배우들 연기력,영상미 액션 굳                   2. 스토리 뭔가 빼먹고 만든거 같음                   3. 킬링 타임용으로 보기 좋음
0.9071679

In [ ]:
#시각화 
pip install pyecharts

In [19]:
#감성분석

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

okt = Okt()    #형태소 분석
tokenizer = Tokenizer(19417, oov_token = 'OOV')
with open('wordIndex.json') as json_file:
  word_index = json.load(json_file)
  tokenizer.word_index = word_index

loaded_model = load_model('best_model.h5')   #감성분석 모델 불러오기
def sentiment_predict(new_sentence):
    print(new_sentence)
    max_len = 30
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    #stopwords: 불용어 리스트 -> 배제
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    return score  #긍정/부정에 대한 점수
    if score >= 0.5:
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score*100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1-score)*100))

        
        
#네이버 영화 리뷰_크롤링

from bs4 import BeautifulSoup
import urllib.request as req

page_num = 1
while True:
    code = req.urlopen("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=189069&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}".format(page_num))
    soup = BeautifulSoup(code, "html.parser")
    comment = soup.select("li > div.score_reple > p > span")
    if len(comment) == 0:
        break
    for i in comment:
        i = i.text.strip()
        if i == "관람객":
            continue

        #print(i)
        result = sentiment_predict(i)
        print(result)
        print("-------------------------------------")
    page_num += 1
    # 양이 많으니 페이지 10개만 
    if page_num == 10:
        break
        

박정민이 왜 포스터, 예고에 없는 지 알았음. 어쩜 그리 연기를 잘 하는지
0.7835091352462769
-------------------------------------
박정민은진짜다박정민은진짜다인정하자
0.6522318720817566
-------------------------------------
목숨과바꾼 가족사랑  그리고 액션 특히 박정민 배우님에 충격적인 변신..좋았어요
0.9959299564361572
-------------------------------------
역시 박정민!!!!조연이아니라 나에겐주연  강철비2보려다 조연으로박정민이나온다해서바로선택한영화인데완전나의신의하수였다?
0.9554305076599121
-------------------------------------
박정민이 가장 강렬했음
0.9572926163673401
-------------------------------------
와 이거 감독판으로도 나왔으면 좋겠다
0.945243775844574
-------------------------------------
역시 믿고 보는 배우 황정민. 이정재.박정민 이영화는 꼭봐야한다!!!올해 최고의 작품 강추
0.9988682270050049
-------------------------------------
황정민이랑 이정재 같이 나오는 영화는 다 재밌어
0.954969048500061
-------------------------------------
몰입감 긴장감으로 시간 순삭ㅋㅋㅋ 존잼
0.9751993417739868
-------------------------------------
스토리가 없어요 정말이에요
0.600499153137207
-------------------------------------
영화 세줄요약 1. 배우들 연기력,영상미 액션 굳                   2. 스토리 뭔가 빼먹고 만든거 같음                   3. 킬링 타임용으로 보기 좋음
0.9071679